## Importing packages

In [1]:
import sys
sys.path.append('/home/class_NI2021/ctxctl_contrib_2023')
import samna
import samna.dynapse1 as dyn1
from dynapse1constants import *
import dynapse1utils as ut
import netgen as n
import params
import time
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pickle
import os

## Looking for available devices

In [2]:
# Checking the list of unopened devices
devices = samna.device.get_unopened_devices()

if len(devices) == 0:
    raise Exception("no device detected!")

for i in range(len(devices)):
    print("["+str(i)+"]: ", devices[i], "serial_number", devices[i].serial_number)

[0]:  Bus 3 Device 4 Dynapse1DevKit serial_number 00000000
[1]:  Bus 1 Device 3 Dynapse1DevKit serial_number 00000031
[2]:  Bus 1 Device 2 Dynapse1DevKit serial_number 00000001


In [3]:
# Select one device from the list
model,no_gui = ut.open_dynapse1(gui=False, sender_port=17654, receiver_port=17523, select_device=True) # returns Dynapse1Model

ERROR: Address already in use, please re-run open_device()!
[0]:  Bus 3 Device 4 Dynapse1DevKit serial_number 00000000
[1]:  Bus 1 Device 3 Dynapse1DevKit serial_number 00000031
[2]:  Bus 1 Device 2 Dynapse1DevKit serial_number 00000001
Sender port: tcp://0.0.0.0:17654
Receiver port: tcp://0.0.0.0:17523
Opened device name: Dynapse1DevKit
SamnaNode ID: 1
PythonNode ID: 2
0 Dynapse1Wrapper created! libcaer init...
Clearing chip 0... DONE.
Clearing chip 1... DONE.
Clearing chip 2... DONE.
Clearing chip 3... DONE.


In [28]:
def build_network(api, n_bits, a_number_bin, b_number_bin):
    # silent all neurons: enforces some biases to ensure neurons are NOT firing
    paramGroup = params.gen_clean_param_group()
    for chip in range(4):
        for core in range(4):
            model.update_parameter_group(paramGroup, chip, core)

    # ---- Spike generator ----
    id_ini = 1
    id_end = id_ini + 1
    spike_generator_ids = np.array(range(id_ini, id_end))
    spike_generators = n.NeuronGroup(0, 0, spike_generator_ids, True)  # Avoid spike generator with ID 0

    # ---- DPI Neurons ----
    core_id = 1

    id_ini = id_end
    if n_bits > 1:
        id_end = id_ini + 5 * (n_bits - 1)
    else:
        id_end = id_ini + 1
    op_delay_pop = n.NeuronGroup(0, core_id, range(id_ini, id_end), False)

    a_delay_pop = []
    for i in range(n_bits - 1):
        id_ini = id_end
        id_end += 2 + 5 * i
        a_delay_pop.append(n.NeuronGroup(0, core_id, range(id_ini, id_end), False))

    b_delay_pop = []
    for i in range(n_bits - 1):
        id_ini = id_end
        id_end += 2 + 5 * i
        b_delay_pop.append(n.NeuronGroup(0, core_id, range(id_ini, id_end), False))

    id_ini = id_end
    id_end += 6
    ha_pop = n.NeuronGroup(0, core_id, range(id_ini, id_end), False)

    if n_bits > 1:
        id_ini = id_end
        id_end += 18 * (n_bits - 1)
        fa_pop = n.NeuronGroup(0, core_id, range(id_ini, id_end), False)

    # ---- Synapses ----
    net_gen = n.NetworkGenerator()

    # OP chain
    net_gen.add_connection(spike_generators.neurons[0], op_delay_pop.neurons[0], dyn1.Dynapse1SynType.AMPA)
    for i in range(1, len(op_delay_pop.neurons)):
        net_gen.add_connection(op_delay_pop.neurons[i - 1], op_delay_pop.neurons[i], dyn1.Dynapse1SynType.AMPA)

    # OP to NOT
    net_gen.add_connection(spike_generators.neurons[0], ha_pop.neurons[0], dyn1.Dynapse1SynType.AMPA)
    net_gen.add_connection(spike_generators.neurons[0], ha_pop.neurons[1], dyn1.Dynapse1SynType.AMPA)
    for i in range(n_bits - 1):
        net_gen.add_connection(op_delay_pop.neurons[1 + 5 * i], fa_pop.neurons[18 * i], dyn1.Dynapse1SynType.AMPA)
        net_gen.add_connection(op_delay_pop.neurons[1 + 5 * i], fa_pop.neurons[18 * i + 1], dyn1.Dynapse1SynType.AMPA)

    # A and B chains
    # -------- ESTO CAMBIA -------
    for i in range(1, n_bits):
        if a_number_bin[len(a_number_bin) - 1 - i] == '1':
            net_gen.add_connection(spike_generators.neurons[0], a_delay_pop[i-1].neurons[0], dyn1.Dynapse1SynType.AMPA)  # A
        if b_number_bin[len(b_number_bin) - 1 - i] == '1':
            net_gen.add_connection(spike_generators.neurons[0], b_delay_pop[i-1].neurons[0], dyn1.Dynapse1SynType.AMPA)  # B
    # ----------------------------

    for i in range(n_bits - 1):
        for j in range(1, len(a_delay_pop[i].neurons)):
            net_gen.add_connection(a_delay_pop[i].neurons[j - 1], a_delay_pop[i].neurons[j], dyn1.Dynapse1SynType.AMPA)
        for j in range(1, len(b_delay_pop[i].neurons)):
            net_gen.add_connection(b_delay_pop[i].neurons[j - 1], b_delay_pop[i].neurons[j], dyn1.Dynapse1SynType.AMPA)

    # Input to NOT
    if a_number_bin[len(a_number_bin) - 1] == '1':
        net_gen.add_connection(spike_generators.neurons[0], ha_pop.neurons[0], dyn1.Dynapse1SynType.GABA_B)  # A
    if b_number_bin[len(b_number_bin) - 1] == '1':
        net_gen.add_connection(spike_generators.neurons[0], ha_pop.neurons[1], dyn1.Dynapse1SynType.GABA_B)  # B
    for i in range(n_bits - 1):
        net_gen.add_connection(a_delay_pop[i].neurons[len(a_delay_pop[i].neurons) - 1], fa_pop.neurons[18 * i], dyn1.Dynapse1SynType.GABA_B)
        net_gen.add_connection(b_delay_pop[i].neurons[len(b_delay_pop[i].neurons) - 1], fa_pop.neurons[18 * i + 1], dyn1.Dynapse1SynType.GABA_B)

    # --------- HA internal structure ----------
    # 2
    net_gen.add_connection(op_delay_pop.neurons[0], ha_pop.neurons[2], dyn1.Dynapse1SynType.AMPA)
    net_gen.add_connection(ha_pop.neurons[0], ha_pop.neurons[2], dyn1.Dynapse1SynType.GABA_B)
    net_gen.add_connection(ha_pop.neurons[1], ha_pop.neurons[2], dyn1.Dynapse1SynType.GABA_B)

    # 3
    net_gen.add_connection(ha_pop.neurons[0], ha_pop.neurons[3], dyn1.Dynapse1SynType.AMPA)
    net_gen.add_connection(ha_pop.neurons[1], ha_pop.neurons[3], dyn1.Dynapse1SynType.GABA_B)

    # 4
    net_gen.add_connection(ha_pop.neurons[0], ha_pop.neurons[4], dyn1.Dynapse1SynType.GABA_B)
    net_gen.add_connection(ha_pop.neurons[1], ha_pop.neurons[4], dyn1.Dynapse1SynType.AMPA)

    # 5
    net_gen.add_connection(ha_pop.neurons[3], ha_pop.neurons[5], dyn1.Dynapse1SynType.AMPA)
    net_gen.add_connection(ha_pop.neurons[4], ha_pop.neurons[5], dyn1.Dynapse1SynType.AMPA)

    # Connecting to the next adder
    if n_bits > 1:
        net_gen.add_connection(ha_pop.neurons[2], fa_pop.neurons[2], dyn1.Dynapse1SynType.AMPA)

    # --------- FA internal structure ----------
    for i in range(n_bits - 1):
        # 3
        net_gen.add_connection(op_delay_pop.neurons[2 + 5 * i], fa_pop.neurons[3 + 18 * i], dyn1.Dynapse1SynType.AMPA)
        net_gen.add_connection(fa_pop.neurons[0 + 18 * i], fa_pop.neurons[3 + 18 * i], dyn1.Dynapse1SynType.GABA_B)
        net_gen.add_connection(fa_pop.neurons[1 + 18 * i], fa_pop.neurons[3 + 18 * i], dyn1.Dynapse1SynType.GABA_B)

        # 4
        net_gen.add_connection(fa_pop.neurons[0 + 18 * i], fa_pop.neurons[4 + 18 * i], dyn1.Dynapse1SynType.AMPA)
        net_gen.add_connection(fa_pop.neurons[1 + 18 * i], fa_pop.neurons[4 + 18 * i], dyn1.Dynapse1SynType.GABA_B)

        # 5
        net_gen.add_connection(fa_pop.neurons[0 + 18 * i], fa_pop.neurons[5 + 18 * i], dyn1.Dynapse1SynType.GABA_B)
        net_gen.add_connection(fa_pop.neurons[1 + 18 * i], fa_pop.neurons[5 + 18 * i], dyn1.Dynapse1SynType.AMPA)

        # 6
        net_gen.add_connection(fa_pop.neurons[2 + 18 * i], fa_pop.neurons[6 + 18 * i], dyn1.Dynapse1SynType.AMPA)

        # 7
        net_gen.add_connection(fa_pop.neurons[3 + 18 * i], fa_pop.neurons[7 + 18 * i], dyn1.Dynapse1SynType.AMPA)

        # 8
        net_gen.add_connection(op_delay_pop.neurons[3 + 5 * i], fa_pop.neurons[8 + 18 * i], dyn1.Dynapse1SynType.AMPA)
        net_gen.add_connection(fa_pop.neurons[4 + 18 * i], fa_pop.neurons[8 + 18 * i], dyn1.Dynapse1SynType.GABA_B)
        net_gen.add_connection(fa_pop.neurons[5 + 18 * i], fa_pop.neurons[8 + 18 * i], dyn1.Dynapse1SynType.GABA_B)

        # 9
        net_gen.add_connection(fa_pop.neurons[4 + 18 * i], fa_pop.neurons[9 + 18 * i], dyn1.Dynapse1SynType.AMPA)
        net_gen.add_connection(fa_pop.neurons[5 + 18 * i], fa_pop.neurons[9 + 18 * i], dyn1.Dynapse1SynType.AMPA)

        # 10
        net_gen.add_connection(fa_pop.neurons[6 + 18 * i], fa_pop.neurons[10 + 18 * i], dyn1.Dynapse1SynType.AMPA)

        # 11
        net_gen.add_connection(op_delay_pop.neurons[3 + 5 * i], fa_pop.neurons[11 + 18 * i], dyn1.Dynapse1SynType.AMPA)
        net_gen.add_connection(fa_pop.neurons[6 + 18 * i], fa_pop.neurons[11 + 18 * i], dyn1.Dynapse1SynType.GABA_B)

        # 12
        net_gen.add_connection(fa_pop.neurons[7 + 18 * i], fa_pop.neurons[12 + 18 * i], dyn1.Dynapse1SynType.AMPA)

        # 13
        net_gen.add_connection(op_delay_pop.neurons[4 + 5 * i], fa_pop.neurons[13 + 18 * i], dyn1.Dynapse1SynType.AMPA)
        net_gen.add_connection(fa_pop.neurons[8 + 18 * i], fa_pop.neurons[13 + 18 * i], dyn1.Dynapse1SynType.GABA_B)
        net_gen.add_connection(fa_pop.neurons[11 + 18 * i], fa_pop.neurons[13 + 18 * i], dyn1.Dynapse1SynType.GABA_B)

        # 14
        net_gen.add_connection(fa_pop.neurons[9 + 18 * i], fa_pop.neurons[14 + 18 * i], dyn1.Dynapse1SynType.AMPA)
        net_gen.add_connection(fa_pop.neurons[10 + 18 * i], fa_pop.neurons[14 + 18 * i], dyn1.Dynapse1SynType.GABA_B)

        # 15
        net_gen.add_connection(fa_pop.neurons[9 + 18 * i], fa_pop.neurons[15 + 18 * i], dyn1.Dynapse1SynType.GABA_B)
        net_gen.add_connection(fa_pop.neurons[10 + 18 * i], fa_pop.neurons[15 + 18 * i], dyn1.Dynapse1SynType.AMPA)

        # 16
        net_gen.add_connection(fa_pop.neurons[12 + 18 * i], fa_pop.neurons[16 + 18 * i], dyn1.Dynapse1SynType.AMPA)
        net_gen.add_connection(fa_pop.neurons[13 + 18 * i], fa_pop.neurons[16 + 18 * i], dyn1.Dynapse1SynType.AMPA)
        # Connecting to the next adder
        if i != (n_bits - 2):
            net_gen.add_connection(fa_pop.neurons[16 + 18 * i], fa_pop.neurons[2 + 18 * (i + 1)], dyn1.Dynapse1SynType.AMPA)

        # 17
        net_gen.add_connection(fa_pop.neurons[14 + 18 * i], fa_pop.neurons[17 + 18 * i], dyn1.Dynapse1SynType.AMPA)
        net_gen.add_connection(fa_pop.neurons[15 + 18 * i], fa_pop.neurons[17 + 18 * i], dyn1.Dynapse1SynType.AMPA)

    # make a dynapse1config using the network: that is convert the validated network to a Dynapse1 configuration
    new_config = net_gen.make_dynapse1_configuration()

    # apply the configuration
    model.apply_configuration(new_config)

    # Initialize custom parameters for a core and a chip:
    paramGroup = dyn1.Dynapse1ParameterGroup() 
    paramGroup.param_map["IF_THR_N"].coarse_value = 7
    paramGroup.param_map["IF_THR_N"].fine_value = 32
    paramGroup.param_map["IF_RFR_N"].coarse_value = 7  # Inverse
    paramGroup.param_map["IF_RFR_N"].fine_value = 255  # Inverse
    paramGroup.param_map["IF_TAU1_N"].coarse_value = 6  # Inverse
    paramGroup.param_map["IF_TAU1_N"].fine_value = 191  # Inverse
    paramGroup.param_map["IF_DC_P"].coarse_value = 0
    paramGroup.param_map["IF_DC_P"].fine_value = 0

    # Fast excitatory synapse (AMPA)
    paramGroup.param_map["NPDPIE_THR_F_P"].coarse_value = 6
    paramGroup.param_map["NPDPIE_THR_F_P"].fine_value =  127
    paramGroup.param_map["NPDPIE_TAU_F_P"].coarse_value = 4
    paramGroup.param_map["NPDPIE_TAU_F_P"].fine_value =  127
    paramGroup.param_map["PS_WEIGHT_EXC_F_N"].coarse_value = 7
    paramGroup.param_map["PS_WEIGHT_EXC_F_N"].fine_value = 255

    # Fast inhibitory synapse (GABA_B)
    paramGroup.param_map["NPDPII_THR_S_P"].coarse_value = 6
    paramGroup.param_map["NPDPII_THR_S_P"].fine_value =  127
    paramGroup.param_map["NPDPII_TAU_S_P"].coarse_value = 4
    paramGroup.param_map["NPDPII_TAU_S_P"].fine_value = 127
    paramGroup.param_map["PS_WEIGHT_INH_S_N"].coarse_value = 7
    paramGroup.param_map["PS_WEIGHT_INH_S_N"].fine_value = 255

    # Apply the custom parameters to the cores
    for i in range(4):
        for j in range(4):
            model.update_parameter_group(paramGroup, i, j)

    return ha_pop, fa_pop


def get_results(op_times, api, monitored_neurons):
    fpga_spike_gen = model.get_fpga_spike_gen()

    spike_times = op_times
    gen_neuron_ids = np.concatenate([[1] * len(op_times) for i in range(len(op_times))], axis=0)  # Spike generator associated to the spike at that index
    spike_times, gen_neuron_ids = map(list, zip(*sorted(zip(spike_times, gen_neuron_ids), reverse=False)))

    post_neuron_chips = [0] * len(gen_neuron_ids)

    isi_base = 900
    repeat_mode = False
    ut.set_fpga_spike_gen(fpga_spike_gen, spike_times, gen_neuron_ids, post_neuron_chips, isi_base, repeat_mode)
    graph, filter_node, sink_node = ut.create_neuron_select_graph(model, monitored_neurons)

    api.reset_timestamp()
    graph.start()
    sink_node.get_events()
    fpga_spike_gen.start()

    if(spike_times[-1] - 0.2 < 0):
        time.sleep(spike_times[-1])
    else:
        time.sleep(spike_times[-1] - 0.2)  # Last spike time - 0.2

    fpga_spike_gen.stop()
    time.sleep(0.5)
    events = sink_node.get_events()
    graph.stop()

    return events



In [53]:
import random

api  = model.get_dynapse1_api()

reps = 3
n_bit_list = [2, 3, 4, 5, 6]
n_factor_list = [0.5, 1, 2, 5, 10]

for i in range(reps):
    for n_bits in n_bit_list:
        for factor in n_factor_list:
            op_times = np.concatenate([np.array(range(10, 40)) / (1000 * factor) + 0.05, [0.400, 0.401]], axis=0)
            a_number = 3
            a_number_bin = format(a_number, "0" + str(n_bits) + "b")
            b_number = 2 ** (n_bits + 1) - 1 
            b_number_bin = format(b_number, "0" + str(n_bits) + "b")
            print(a_number_bin, b_number_bin)

            ha_pop, fa_pop = build_network(api, n_bits, a_number_bin, b_number_bin)
                    
            if n_bits > 1:
                monitored_neurons = np.concatenate([ha_pop.tuple_neuron_ids, fa_pop.tuple_neuron_ids], axis=0)
            else:
                monitored_neurons = np.array(ha_pop.tuple_neuron_ids)

            events = get_results(op_times, api, monitored_neurons)
            evts_n = np.array([[evt.timestamp / 10**6 + op_times[0], evt.neuron_id] for evt in events])

            save_array = [op_times, a_number_bin, b_number_bin, evts_n]
            folder = "results_ADDER_single_op/"
            test_name = "adder_2_inputs_" + str(n_bits) + "bits_factor" + str(factor)

            cwd = os.getcwd()
            if not os.path.exists(cwd + "/" + folder):
                os.mkdir(cwd + "/" + folder)

            i = 1
            while os.path.exists(cwd + "/" + folder + test_name + "_" + str(i) + ".pickle"):
                i += 1

            filename = test_name + "_" + str(i)

            with open(folder + filename + '.pickle', 'wb') as handle:
                pickle.dump(save_array, handle, protocol=pickle.HIGHEST_PROTOCOL)
            with open(folder + filename + '.pickle', "rb") as handle:
                print("Loading... OK")

            plt.rcParams['figure.dpi'] = 400
            plt.rcParams['font.size'] = '4'
            if n_bits > 1:
                plt.rcParams["figure.figsize"] = (4, 0.8 * (7 + 3 * (n_bits - 2)) / 7)
            else:
                plt.rcParams["figure.figsize"] = (4, 0.8)

            plt.plot(np.array(op_times), [-1] * len(op_times), 'o', markersize=0.5, color='tab:blue')
            for i in range(len(op_times)):
                for j in range(n_bits):
                    if a_number_bin[len(a_number_bin) - 1 - j] == '1':
                        plt.plot(op_times[i] , j, 'o', markersize=0.5, color='tab:orange')
                    if b_number_bin[len(b_number_bin) - 1 - j] == '1':
                        plt.plot(op_times[i], j + n_bits, 'o', markersize=0.5, color='tab:olive')

            plt.xlabel('Simulation time (s)')
            plt.yticks(range(-1, n_bits * 3))
            plt.gca().set_yticklabels(["OP"] + ["A" + str(i) for i in range(n_bits)] + ["B" + str(i) for i in range(n_bits)] + ["S" + str(i) for i in range(n_bits)])
            plt.ylabel("Neurons")
            plt.xlim([op_times[0] - 0.001 / factor, op_times[-3] + 0.025 / factor])
            plt.ylim([-2, n_bits * 3])

            evts_ha = np.array([[evt.timestamp / 10**6 + op_times[0], evt.neuron_id] for evt in events if evt.neuron_id == ha_pop.neurons[5].neuron_id])
            if len(evts_ha) != 0:
                plt.plot(evts_ha[:,0], [n_bits * 2] * len(evts_ha[:,0]), 'o', markersize=0.5, color='tab:green')

            evts_fa = []
            for i in range(n_bits - 1):
                evts_tmp = []
                for evt in events:
                    if evt.neuron_id == fa_pop.neurons[17 + 18 * i].neuron_id:
                        evt_info = [evt.timestamp / 10**6 + op_times[0], evt.neuron_id]
                        evts_fa.append(evt_info)
                        evts_tmp.append(evt_info)

                evts_tmp = np.array(evts_tmp)
                if len(evts_tmp) != 0:
                    plt.plot(evts_tmp[:,0], [n_bits * 2 + 1 + i] * len(evts_tmp[:,0]), 'o', markersize=0.5, color='tab:green')

            plt.tight_layout()
            plt.savefig(folder + filename + '.png', transparent=False, facecolor='white', edgecolor='black')
            plt.clf()
            
            time.sleep(0.5)

11 111
New configuration applied to DYNAP-SE1!
VariableIsiMode already 1
RepeatMode already 0
Loading... OK
11 111
New configuration applied to DYNAP-SE1!
VariableIsiMode already 1
RepeatMode already 0
Loading... OK


/tmp/ipykernel_132379/262988965.py:87: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


11 111
New configuration applied to DYNAP-SE1!
VariableIsiMode already 1
RepeatMode already 0
Loading... OK
11 111
New configuration applied to DYNAP-SE1!
VariableIsiMode already 1
RepeatMode already 0
Loading... OK
11 111
New configuration applied to DYNAP-SE1!
VariableIsiMode already 1
RepeatMode already 0
Loading... OK
011 1111
New configuration applied to DYNAP-SE1!
VariableIsiMode already 1
RepeatMode already 0
Loading... OK
011 1111
New configuration applied to DYNAP-SE1!
VariableIsiMode already 1
RepeatMode already 0
Loading... OK
011 1111
New configuration applied to DYNAP-SE1!
VariableIsiMode already 1
RepeatMode already 0
Loading... OK
011 1111
New configuration applied to DYNAP-SE1!
VariableIsiMode already 1
RepeatMode already 0
Loading... OK
011 1111
New configuration applied to DYNAP-SE1!
VariableIsiMode already 1
RepeatMode already 0
Loading... OK
0011 11111
New configuration applied to DYNAP-SE1!
VariableIsiMode already 1
RepeatMode already 0
Loading... OK
0011 11111
New

<Figure size 1600x320 with 0 Axes>

## Close the device

In [55]:
# remeber to close the device
samna.device.close_device(model)